## Calculation of Circularly Polarized Light Absorption in Two-Dimensional Systems





Steps:
1. Open a directory inside the ‘regular’, linearly polarized light calculation you finished and move inside it.

In [ ]:
mkdir cpl_abs
cd cpl_abs

2. Link the eigenvectors.h5 file

In [ ]:
ln -s ../eigenvectors.h5 .

3. Open a directory for the x-polarized light and for the y-polarized light:

In [ ]:
mkdir x
mkdir y

4. Run the python script that calculates the polarization directions for x and for y:

In [ ]:
python vmtxel_x_y.py .

You got the polarization vector (its size doesn’t matter, it is renormalized in the code anyway) you should use to get vmtxel along x and along y.


5. Go inside x/y, copy absorption.inp and the job file from the ‘regular’ absorption calculation.
6. Add the ‘noeh_only’ keyword since you do not need to calculate the entire absorption to generate vmtxel, which is what we are interested in now.
7. Add ‘polarization B1 B2 0.0’, where B1 and B2 are printed from the python script you previously ran (calculate_vmtxel_x_y.py). 

notice that you cannot use 'eqp_co_corrections' together with 'noeh_only'
 


In [ ]:
cd x
cp ../absorption.inp ../job.absorption .
echo 'noeh_only' >> absorption.inp
echo 'polarization B1x B2x 0.0' >> absorption.inp

cd ../y
cp ../absorption.inp ../job.absorption .
echo 'noeh_only' >> absorption.inp
echo 'polarization B1y B2y 0.0' >> absorption.inp

cd ..

   
8. Run the absorption calculation for x and for y.
    You might not need one of them if one of the reciprocal lattice vectors is already along x or along y (or two of them if the lattice vectors are perpendicular).
    You can always calculate only one more direction, perpendicular to on of the directions you previously calculated, but for simplicity we calculate vmtxel exactly along x and along y.

In [ ]:
cd x
qsub job.absorption
cd ../y
qsub job.absorption
cd ..

9. Create another folder inside cpl_abs for the calculation of the vmtxel along the circularly polarized light directions and link the vmtxel elements along x and y that you calculated.


In [ ]:
mkdir vmtxel_sigma
cd vmtxel_sigma
ln -s ../x/vmtxel vmtxel_x_bin
ln -s ../y/vmtxel vmtxel_y_bin

If you do not calculate one of the directions since they were already calculated in the ‘regular’ absorption run, link the file you were instructed to by the python script and make sure it has the name vmtxel_x_bin or vmtxel_y_bin.

10. Convert the x and y vmtxel bin files to bin format:


In [ ]:
/BGW_dir/bin/bsebinasc.cplx.x _x_bin _x_ascii
/BGW_dir/bin/bsebinasc.cplx.x _y_bin _y_ascii

11. Run the python script that calculates the vmtxel files along sigma+ and along sigma-

This script calculates the dipole matrix elements along the two circulalrly polarized light directions, using the matrix elements calculated along the x and y directions following:

$V_{\sigma^+}=\frac{V_{x}+iV_{y}}{\sqrt{2}}$

$V_{\sigma^-}=\frac{V_{x}-iV_{y}}{\sqrt{2}}$

In [ ]:
python vmtxel_sigma_pm.py

12. Convert the calculated vmtxel files from ascii format back to bin format:

In [ ]:
/BGW_dir/bin/bseascbin.cplx.x _sigma_plus_ascii _sigma_plus_bin
/BGW_dir/bin/bseascbin.cplx.x _sigma_minus_ascii _sigma_minus_bin

13. Go one folder up, create a directory for the circularly polarized light absorption. The BSE Hamiltonian and its diagonalization & eigenvectors do not depend on the polarization direction, so you might not really need to do this if you want to process the dipole matrix elements yourself.

In [ ]:
cd ..
mkdir cpl_abs_calc
cd cpl_abs_calc

14. Copy the job file, absorption input file and link the files you previously linked for the initial absorption run. You should also link the vmtxel files you calculated for the circularly polarized light directions. You must also specify a third direction, so you can use the out-of-plane direction of the 2D system which should be the b3 direction in the initial run.

In [ ]:
ln -s ../vmtxel_sigma/vmtxel_sigma_plus_bin vmtxel_b1
ln -s ../vmtxel_sigma/vmtxel_sigma_plus_bin vmtxel_b2
ln -s ../vmtxel_b3 .
cp ../absorption.inp ../job.absorption .
echo 'read_vmtxel' >> absorption.inp


15. submit the absorption job file for the circulalrly polarized light.

In [ ]:
qsub job.absorption